# Install detectron2

In [1]:
# install dependencies: 
!pip install pyyaml==5.1
!pip install torch==1.8.1 torchvision==0.9.1 torchtext===0.9.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!find . -name ".DS_Store" -delete

     |████████████████████████████████| 274 kB 7.4 MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp37-cp37m-linux_x86_64.whl size=44091 sha256=488a24ea2ac78ef9f7d0df64fb5fb8c258eb45789f0ade07760b5f25e40079bf
  Stored in directory: /root/.cache/pip/wheels/77/f5/10/d00a2bd30928b972790053b5de0c703ca87324f3fead0f2fd9
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 804.1 MB 2.8 kB/s 
     |████████████████████████████████| 17.4 MB 1.4 MB/s 
     |████████████████████████████████| 7.1 MB 26.7 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.10.0+cu102
    Uninstalling torchvision-0.10.0+cu102:
      Successfully unins

In [2]:
# install detectron2: (Colab has CUDA 10.1 + torch 1.8)
import torch
assert torch.__version__.startswith("1.8")   # need to manually install torch 1.8 if Colab changes its default version
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.8/index.html
     |████████████████████████████████| 6.3 MB 1.7 MB/s 
     |████████████████████████████████| 74 kB 2.7 MB/s 
     |████████████████████████████████| 144 kB 11.9 MB/s 
     |████████████████████████████████| 130 kB 12.0 MB/s 
     |████████████████████████████████| 49 kB 5.8 MB/s 
     |████████████████████████████████| 722 kB 12.5 MB/s 
     |████████████████████████████████| 743 kB 19.8 MB/s 
     |████████████████████████████████| 112 kB 38.2 MB/s 
  Created wheel for fvcore: filename=fvcore-0.1.5.post20210804-py3-none-any.whl size=60618 sha256=d226dae64ee853db8f89bcf6b7f5d40197c16689443425de4d4201e906f82052
  Stored in directory: /root/.cache/pip/wheels/1d/e2/fe/67887e71552be741faccead8f7a8e013b6e0b1225cf591afa1
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141229 sha256=874731f3ff3157126806c5ca14e195f09219dea4f943f0e8e315277ec7b360

In [3]:
# Basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

# Prepare the dataset


In [4]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive')

Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [5]:
!rm -rf train 
!rm -rf validation 
!rm -rf test

In [6]:
# Unzip the training and test dataset quietly
!echo "Unzipping training set"
!unzip -q '/content/drive/MyDrive/Colab Notebooks/datasets/visdrone/train.zip' -d '/content'
!echo "Done!"

!echo "Unzipping validation set"
!unzip -q '/content/drive/MyDrive/Colab Notebooks/datasets/visdrone/validation.zip' -d '/content'
!echo "Done!"

!echo "Unzipping test set"
!unzip -q '/content/drive/MyDrive/Colab Notebooks/datasets/visdrone/test.zip' -d '/content'
!echo "Done!"

Unzipping training set
Done!
Unzipping validation set
Done!
Unzipping test set
Done!


In [7]:
from detectron2.data.datasets import register_coco_instances

#Register the training and test custom datasets (COCO format)
if 'visdrone_train' in DatasetCatalog.list():
  DatasetCatalog.remove("visdrone_train")
if 'visdrone_validation' in DatasetCatalog.list():
  DatasetCatalog.remove("visdrone_validation")
if 'visdrone_test' in DatasetCatalog.list():
  DatasetCatalog.remove("visdrone_test")

register_coco_instances("visdrone_train", {}, "/content/train/annotations.json", "/content/train/images")
register_coco_instances("visdrone_validation", {}, "/content/validation/annotations.json", "/content/validation/images")
register_coco_instances("visdrone_test", {}, "/content/test/annotations.json", "/content/test/images")

Verify that the training dataset loaded correctly. Visualise a few random images with their ground truths.


In [8]:
train_dataset_dicts = DatasetCatalog.get("visdrone_train")
train_metadata = MetadataCatalog.get("visdrone_train")

[08/10 05:16:06 d2.data.datasets.coco]: Loading /content/train/annotations.json takes 2.00 seconds.
[08/10 05:16:07 d2.data.datasets.coco]: Loaded 6471 images in COCO format from /content/train/annotations.json


Verify if validation dataset loads correctly. Visualise a few random images with their ground truths.


In [9]:
validation_dataset_dicts = DatasetCatalog.get("visdrone_validation")
validation_metadata = MetadataCatalog.get("visdrone_validation")

[08/10 05:16:08 d2.data.datasets.coco]: Loaded 548 images in COCO format from /content/validation/annotations.json


## Training

Now, let's fine-tune a COCO-pretrained R50-C4 Faster R-CNN model on the training dataset.

In [39]:
from detectron2.engine import HookBase
from detectron2.data import build_detection_train_loader
from detectron2.engine import DefaultTrainer
import detectron2.utils.comm as comm

# Get default config for the specified model from Model Zoo, customize certain parameters
_config_file = "COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file(_config_file))
cfg.DATASETS.TRAIN = ("visdrone_train",)
cfg.DATASETS.VAL = ("visdrone_validation",)
cfg.DATASETS.TEST = ("visdrone_validation",) 

# Add a custom hook to measure loss periodically on a validation dataset. 
class ValidationLoss(HookBase):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg.clone()
        self.cfg.DATASETS.TRAIN = cfg.DATASETS.VAL
        self._loader = iter(build_detection_train_loader(self.cfg))
        
    def after_step(self):
        data = next(self._loader)
        with torch.no_grad():
            loss_dict = self.trainer.model(data)
            
            losses = sum(loss_dict.values())
            assert torch.isfinite(losses).all(), loss_dict

            loss_dict_reduced = {"val_" + k: v.item() for k, v in 
                                 comm.reduce_dict(loss_dict).items()}
            losses_reduced = sum(loss for loss in loss_dict_reduced.values())
            if comm.is_main_process():
                self.trainer.storage.put_scalars(total_val_loss=losses_reduced, 
                                                 **loss_dict_reduced)

# Rest of the config
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url(_config_file)  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 12000
cfg.TEST.EVAL_PERIOD = 100
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 12

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
val_loss = ValidationLoss(cfg)  
trainer.register_hooks([val_loss])
# swap the order of PeriodicWriter and ValidationLoss
trainer._hooks = trainer._hooks[:-2] + trainer._hooks[-2:][::-1]

[08/10 06:17:28 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

## Inference & evaluation using the trained model
Now, let's run inference with the trained model on the balloon validation dataset. First, let's create a predictor using the model we just trained:



In [40]:
# Load trained models
!rm ./output/model_final.pth
!rm ./output/last_checkpoint
!cp '/content/drive/MyDrive/Colab Notebooks/models/fpn50/model_final.pth' ./output/model_final.pth
!cp '/content/drive/MyDrive/Colab Notebooks/models/fpn50/last_checkpoint' ./output/last_checkpoint

In [41]:
from detectron2.modeling import build_model
model = build_model(cfg)

from detectron2.checkpoint import DetectionCheckpointer
DetectionCheckpointer(model).load("./output/model_final.pth") 

{'iteration': 11999,
 'trainer': {'hooks': {'LRScheduler': {'base_lrs': [0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.00025,
     0.0

In [42]:
# Inference should use the config with parameters that are used in training
# cfg now already contains everything we've set previously. We changed it a little bit for inference:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6   # set a custom testing threshold
predictor = DefaultPredictor(cfg)


Then, we randomly select several samples to visualize the prediction results

In [ ]:
from detectron2.utils.visualizer import ColorMode

test_dataset_dicts = DatasetCatalog.get("visdrone_test")
test_metadata = MetadataCatalog.get("visdrone_test")

indices = [30,780, 556, 778]
for i in indices:    
    d = test_dataset_dicts[i]
    im = cv2.imread(d["file_name"])
    outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
    v = Visualizer(im[:, :, ::-1],
                   metadata=test_metadata, 
                   scale=1, 
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    cv2_imshow(out.get_image()[:, :, ::-1])

In [43]:
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

evaluator = COCOEvaluator("visdrone_test", ("bbox",), False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "visdrone_test")
print(inference_on_dataset(model, val_loader, evaluator))

[08/10 06:17:42 d2.data.datasets.coco]: Loaded 1610 images in COCO format from /content/test/annotations.json
[08/10 06:17:42 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[08/10 06:17:42 d2.data.common]: Serializing 1610 elements to byte tensors and concatenating them all ...
[08/10 06:17:42 d2.data.common]: Serialized dataset takes 2.56 MiB
[08/10 06:17:42 d2.evaluation.evaluator]: Start inference on 1610 batches
[08/10 06:17:44 d2.evaluation.evaluator]: Inference done 11/1610. Dataloading: 0.0040 s/iter. Inference: 0.1097 s/iter. Eval: 0.0005 s/iter. Total: 0.1141 s/iter. ETA=0:03:02
[08/10 06:17:49 d2.evaluation.evaluator]: Inference done 57/1610. Dataloading: 0.0024 s/iter. Inference: 0.1083 s/iter. Eval: 0.0004 s/iter. Total: 0.1112 s/iter. ETA=0:02:52
[08/10 06:17:54 d2.evaluation.evaluator]: Inference done 102/1610. Dataloading: 0.0023 s/iter. Inference: 0.1090 s